The skeleton for the cut callback functions

In [ ]:
from gurobipy import GRB, quicksum

In [ ]:
def mincut_callback(model, where):

    if model._impcounter < 10 and where == GRB.Callback.MIPNODE:
        if model.cbGet(GRB.Callback.MIPNODE_NODCNT) != 0: return
        
        relaxation_objval = model.cbGet(GRB.Callback.MIPNODE_OBJBND)
        if model._relobj is not None:
            imp = (model._relobj - relaxation_objval) / model._relobj
            if imp < 0.005: 
                model._impcounter += 1
            else:
                model._impcounter = 0
        model._relobj = relaxation_objval

        for i in range(model._k):
            capacities = model.cbGetNodeRel(model._vars[i])
            cutsets = model._cutfinder.get_cutsets(capacities)
            for (u, v, cutset) in cutsets:
                cutset_expr = quicksum(model._vars[i][j] for j in cutset)
                model.cbCut(cutset_expr >= model._vars[i][u] + model._vars[i][v] - 1)
        
    elif where == GRB.Callback.MIPSOL:
        for i in range(model._k):
            capacities = model.cbGetSolution(model._vars[i])
            cutsets = model._cutfinder.get_cutsets(capacities)
            for (u, v, cutset) in cutsets:
                cutset_expr = quicksum(model._vars[i][j] for j in cutset)
                model.cbLazy(cutset_expr >= model._vars[i][u] + model._vars[i][v] - 1)
